# Graph Examples

Port-forward to that ingress on localhost:8003 in a separate terminal either with:

  * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:8080`
  * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:8080`


In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-seldon" modified.


## Input and output transformer with model in same pod

In [3]:
!cat seldon-core/examples/graph_examples/tin-model-tout.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: test
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: input-transformer
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: model
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: output-transformer
    graph:
      name: output-transformer
      type: OUTPUT_TRANSFORMER
      endpoint:
        type: GRPC
      children:
        - name: input-transformer
          type: TRANSFORMER
          endpoint:
            type: GRPC
          children:
            - name: model
              type: MODEL
              endpoint:
                type: GRPC
    name: test
    replicas: 1


In [4]:
!kubectl create -f seldon-core/examples/graph_examples/tin-model-tout.yaml

seldondeployment.machinelearning.seldon.io/test created


In [5]:
!kubectl wait --for condition=ready --timeout=300s sdep --all -n seldon

seldondeployment.machinelearning.seldon.io/sklearn condition met
seldondeployment.machinelearning.seldon.io/test condition met


In [6]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8004/seldon/seldon/test/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[1.0,2.0,5.0]]},"meta":{"requestPath":{"input-transformer":"seldonio/metadata-generic-node:1.14.0-dev","model":"seldonio/metadata-generic-node:1.14.0-dev","output-transformer":"seldonio/metadata-generic-node:1.14.0-dev"}}}


In [7]:
!kubectl delete -f tin-model-tout.yaml

error: the path "tin-model-tout.yaml" does not exist


## Input and output transformer with model in separate pods

In [8]:
!cat seldon-core/examples/graph_examples/tin-model-tout-sep-pods.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: test
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: input-transformer
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: model
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: output-transformer
    graph:
      name: output-transformer
      type: OUTPUT_TRANSFORMER
      endpoint:
        type: GRPC
      children:
        - name: input-transformer
          type: TRANSFORMER
          endpoint:
            type: GRPC
          children:
            - name: model
              type: MODEL
              endpoint:
                type: GRPC
    name: test
    replicas: 1


In [9]:
!kubectl create -f seldon-core/examples/graph_examples/tin-model-tout-sep-pods.yaml

Error from server (AlreadyExists): error when creating "seldon-core/examples/graph_examples/tin-model-tout-sep-pods.yaml": seldondeployments.machinelearning.seldon.io "test" already exists


In [10]:
!kubectl wait --for condition=ready --timeout=300s sdep --all -n seldon

seldondeployment.machinelearning.seldon.io/sklearn condition met
seldondeployment.machinelearning.seldon.io/test condition met


In [11]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8004/seldon/seldon/test/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[1.0,2.0,5.0]]},"meta":{"requestPath":{"input-transformer":"seldonio/metadata-generic-node:1.14.0-dev","model":"seldonio/metadata-generic-node:1.14.0-dev","output-transformer":"seldonio/metadata-generic-node:1.14.0-dev"}}}


In [13]:
!kubectl delete -f seldon-core/examples/graph_examples/tin-model-tout-sep-pods.yaml

seldondeployment.machinelearning.seldon.io "test" deleted


## Input and output transformer with svcOrch in separate pod

In [14]:
!cat seldon-core/examples/graph_examples/tin-model-tout-sep-svcorch.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: test
spec:
  annotations:
    seldon.io/engine-separate-pod: "true"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: input-transformer
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: model
        - image: seldonio/metadata-generic-node:1.14.0-dev
          name: output-transformer
    graph:
      name: output-transformer
      type: OUTPUT_TRANSFORMER
      endpoint:
        type: GRPC
      children:
        - name: input-transformer
          type: TRANSFORMER
          endpoint:
            type: GRPC
          children:
            - name: model
              type: MODEL
              endpoint:
                type: GRPC
    name: test
    replicas: 1


In [15]:
!kubectl create -f seldon-core/examples/graph_examples/tin-model-tout-sep-svcorch.yaml

seldondeployment.machinelearning.seldon.io/test created


In [16]:
!kubectl wait --for condition=ready --timeout=300s sdep --all -n seldon

seldondeployment.machinelearning.seldon.io/sklearn condition met


In [ ]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8004/seldon/seldon/test/api/v1.0/predictions \
   -H "Content-Type: application/json"

In [ ]:
!kubectl delete -f seldon-core/examples/graph_examples/tin-model-tout-sep-svcorch.yaml

## Combiner

In [ ]:
!cat combiner.yaml

In [ ]:
!kubectl create -f combiner.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s sdep --all -n seldon

In [ ]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/test/api/v1.0/predictions \
   -H "Content-Type: application/json"

In [ ]:
!kubectl delete -f combiner.yaml